# Managing Kleio files

> First time use: follow instructions in the [README.md](README.md) file in this directory.

## Initial setup

Create a TimelinkNotebook object. This will be the interface for most of your interactions with Timelink.

The first time you run this notebook, you will be prompted to install ìpykernel. This is required to run Timelink in a notebook.

Note:
* First time run takes a little time as the required Docker images are downloaded.
* Timelink will default to using sqlite as the database, see [Receipts notebook for more control](1-receipts.ipynb)


In [1]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()
tlnb.print_info(show_token=True)


Timelink version: 1.1.10
Project name: dehergne-locations
Project home: /Users/jrc/mhk-home/sources/dehergne-locations
Database type: sqlite
Database name: dehergne_locations
Kleio image: timelinkserver/kleio-server
Kleio server token: r9dGv8RTbqOcWUrP34a52jSXr6Evtjz3
Kleio server URL: http://127.0.0.1:8088
Kleio server home: /Users/jrc/mhk-home/sources/dehergne-locations
Kleio server container: brave_cannon
Kleio version requested: latest
Kleio server version: 12.5.570 (2024-02-29 07:39:03)
SQLite directory: /Users/jrc/mhk-home/sources/dehergne-locations/database/sqlite
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=dehergne-locations, project_home=/Users/jrc/mhk-home/sources/dehergne-locations, db_type=sqlite, db_name=dehergne_locations, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


TimelinkNotebook created a database to store data and a Kleio server to 
translate transcriptions in Kleio notation into data that can be imported
into a database.

To get information about the database and the Kleio server do ```tnlb.print_info()```

### Database status

Count the number of rows in each table in the database.


In [2]:
tlnb.table_row_count_df()

,table,count
0,acts,29
1,aregisters,1
2,attributes,25930
3,class_attributes,70
4,classes,14
5,entities,32087
6,geoentities,359
7,goods,0
8,kleiofiles,60
9,objects,0


### Show the kleio files available 

In [74]:
kleio_files = tlnb.get_kleio_files()
kleio_files

,path,name,modified,status,translated,errors,warnings,import_status,import_errors,import_warnings,import_error_rpt,import_warning_rpt,imported,rpt_url,xml_url
0,sources/dehergne-0-abrev.cli,dehergne-0-abrev.cli,2024-05-30 11:26:49.472100+00:00,V,2024-05-30 11:15:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-30 13:11:52.698545,/rest/reports/sources/dehergne-0-abrev.rpt,/rest/exports/sources/dehergne-0-abrev.xml
1,sources/dehergne-a.cli,dehergne-a.cli,2024-06-01 05:19:35.687600+00:00,T,2024-05-31 16:01:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-31 16:02:06.474862,/rest/reports/sources/dehergne-a.rpt,/rest/exports/sources/dehergne-a.xml
2,sources/dehergne-b.cli,dehergne-b.cli,2024-05-31 07:11:25.904181+00:00,V,2024-05-31 03:59:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-31 07:14:02.281938,/rest/reports/sources/dehergne-b.rpt,/rest/exports/sources/dehergne-b.xml
3,sources/dehergne-c.cli,dehergne-c.cli,2024-05-31 07:23:47.445653+00:00,V,2024-05-31 07:23:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-31 07:24:00.842583,/rest/reports/sources/dehergne-c.rpt,/rest/exports/sources/dehergne-c.xml
4,sources/dehergne-d.cli,dehergne-d.cli,2024-05-31 07:11:25.924324+00:00,V,2024-05-31 03:59:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-31 07:14:16.822487,/rest/reports/sources/dehergne-d.rpt,/rest/exports/sources/dehergne-d.xml
5,sources/dehergne-e.cli,dehergne-e.cli,2024-05-30 11:26:49.496012+00:00,V,2024-05-30 11:15:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-30 13:12:22.236161,/rest/reports/sources/dehergne-e.rpt,/rest/exports/sources/dehergne-e.xml
6,sources/dehergne-f.cli,dehergne-f.cli,2024-05-31 07:11:25.931827+00:00,V,2024-05-31 04:00:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-31 07:14:23.426420,/rest/reports/sources/dehergne-f.rpt,/rest/exports/sources/dehergne-f.xml
7,sources/dehergne-g.cli,dehergne-g.cli,2024-06-01 04:42:17.488444+00:00,T,2024-06-01 04:34:00+00:00,0,0,I,0,0,No errors,No warnings,2024-06-01 04:34:35.977856,/rest/reports/sources/dehergne-g.rpt,/rest/exports/sources/dehergne-g.xml
8,sources/dehergne-h.cli,dehergne-h.cli,2024-05-30 11:26:49.503791+00:00,V,2024-05-30 11:15:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-30 13:12:37.569205,/rest/reports/sources/dehergne-h.rpt,/rest/exports/sources/dehergne-h.xml
9,sources/dehergne-i.cli,dehergne-i.cli,2024-05-31 07:11:25.951034+00:00,V,2024-05-31 04:00:00+00:00,0,0,I,0,0,No errors,No warnings,2024-05-31 07:14:31.415438,/rest/reports/sources/dehergne-i.rpt,/rest/exports/sources/dehergne-i.xml


## Optionally clean translations 

In [10]:
tlnb.kleio_server.translation_clean("", recurse="yes")

['sources/dehergne-z.cli',
 'sources/dehergne-y.cli',
 'sources/dehergne-x.cli',
 'sources/dehergne-w.cli',
 'sources/dehergne-v.cli',
 'sources/dehergne-u.cli',
 'sources/dehergne-t.cli',
 'sources/dehergne-s.cli',
 'sources/dehergne-r.cli',
 'sources/dehergne-q.cli',
 'sources/dehergne-p.cli',
 'sources/dehergne-other.cli',
 'sources/dehergne-o.cli',
 'sources/dehergne-n.cli',
 'sources/dehergne-m.cli',
 'sources/dehergne-locations-1701.cli',
 'sources/dehergne-locations-1644.cli',
 'sources/dehergne-l.cli',
 'sources/dehergne-k.cli',
 'sources/dehergne-j.cli',
 'sources/dehergne-i.cli',
 'sources/dehergne-h.cli',
 'sources/dehergne-g.cli',
 'sources/dehergne-f.cli',
 'sources/dehergne-e.cli',
 'sources/dehergne-d.cli',
 'sources/dehergne-c.cli',
 'sources/dehergne-b.cli',
 'sources/dehergne-a.cli',
 'sources/dehergne-0-abrev.cli',
 'identifications/mhk_identification_toliveira.cli']


# Update the database from Kleio sources.

Updates source translations and imports into database sources with no errors.

Only changed files since last import, or new files, will be processed.

In [18]:
import logging
logging.basicConfig(level=logging.INFO)

import threading

# create a thread object that will run the 'tlnb.update_from_sources' function in the background
background_thread = threading.Thread(target=tlnb.update_from_sources)

# start the thread
background_thread.start()



INFO:root:Request translation of T sources/dehergne-p.cli
INFO:root:Importing sources/dehergne-p.cli
INFO:root:Storing 31 postponed relations


In [19]:

#tlnb.update_from_sources()
# get the import status
import_status = tlnb.get_import_status()
# list those with import_error > 0 or error > 0
need_attention = import_status.query("import_status != 'I'")
if len(need_attention) > 0:
    for i, need_attention in need_attention.iterrows():
        print(f"{need_attention.name} tstatus:{need_attention.status} istatus:{need_attention.import_status}")
else:
    print("All files imported successfully!")
if background_thread.is_alive():
    print("Thread is still running")
else:
    print("Thread has finished")

19 tstatus:P istatus:N
Thread is still running


## Check the status of the files

Check the import status of the translated files

* I: Imported
* E: Imported with error
* W: Imported with warnings no errors
* N: Not imported
* U: Translation updated need to reimport

In [89]:
imported_files_df = tlnb.get_import_status()
imported_files_df.info()
imported_files_df[["import_status", "import_errors","import_warnings","name","imported","errors","warnings","path"]].sort_values("name")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   path                31 non-null     object             
 1   name                31 non-null     object             
 2   size                31 non-null     int64              
 3   directory           31 non-null     object             
 4   modified            31 non-null     datetime64[ns, UTC]
 5   modified_iso        31 non-null     datetime64[ns, UTC]
 6   modified_string     31 non-null     object             
 7   qtime               31 non-null     datetime64[ns, UTC]
 8   qtime_string        31 non-null     object             
 9   source_url          31 non-null     object             
 10  status              31 non-null     object             
 11  translated          31 non-null     datetime64[ns, UTC]
 12  translated_string   31 non-null     ob

,import_status,import_errors,import_warnings,name,imported,errors,warnings,path
0,I,0,0,dehergne-0-abrev.cli,2024-05-30 13:11:52.698545,0,0,sources/dehergne-0-abrev.cli
1,I,0,0,dehergne-a.cli,2024-06-01 06:14:24.623391,0,0,sources/dehergne-a.cli
2,I,0,0,dehergne-b.cli,2024-05-31 07:14:02.281938,0,0,sources/dehergne-b.cli
3,I,0,0,dehergne-c.cli,2024-05-31 07:24:00.842583,0,0,sources/dehergne-c.cli
4,I,0,0,dehergne-d.cli,2024-05-31 07:14:16.822487,0,0,sources/dehergne-d.cli
5,I,0,0,dehergne-e.cli,2024-05-30 13:12:22.236161,0,0,sources/dehergne-e.cli
6,I,0,0,dehergne-f.cli,2024-05-31 07:14:23.426420,0,0,sources/dehergne-f.cli
7,I,0,0,dehergne-g.cli,2024-06-01 06:14:31.574148,0,0,sources/dehergne-g.cli
8,I,0,0,dehergne-h.cli,2024-05-30 13:12:37.569205,0,0,sources/dehergne-h.cli
9,I,0,0,dehergne-i.cli,2024-05-31 07:14:31.415438,0,0,sources/dehergne-i.cli


In [91]:

errors = imported_files_df[imported_files_df.import_errors>0][["import_status", "import_errors","import_warnings","name","imported","errors","warnings","path"]].sort_values("name")
errors

,import_status,import_errors,import_warnings,name,imported,errors,warnings,path


### Check a translation report

If needed to check for errors.


In [92]:
file_number=list(errors.index.unique())
if len(file_number) > 0:
    rpt = tlnb.get_translation_report(imported_files_df, file_number)
    print(rpt)
else:
    print("No errors found")



No errors found


### Get the import report for a file

In case there are errors in the import phase check the import report.

In [ ]:
rpt = tlnb.get_import_rpt(imported_files_df, rows=[0])
print(rpt)

No errors
No warnings
